In [1]:
import numpy as np
import pandas as pd
import os
import copy
from tqdm import tqdm
from datetime import datetime, timedelta

# 0 - Initialization

In [2]:
## Input Data
current_year = 2023
path = 'IR Victor/'

## Get the Complete Extract
extract = pd.DataFrame()
for year in range(2019, current_year):
    year_path = path + 'IR ' + str(year) + '/'
    year_extract = [i for i in os.listdir(year_path) if i.startswith('moviment')][0]
    year_extract = pd.read_excel(year_path + year_extract, index_col=1)
    extract = extract.append(year_extract[::-1])

## Manipulate Extract
extract = extract[extract.Produto != 'Tesouro Selic 2025']
extract.Produto = extract.Produto.apply(lambda x: x[:6].split()[0])
extract.Quantidade = extract.Quantidade.apply(lambda x: x.replace(',','.') if not isinstance(x, float) else x).apply(float).apply(np.floor).apply(int)
extract.index = pd.to_datetime(extract.index, format='%d/%m/%Y')
# Work around to consider the data from a subscription (not given in the Excel data)
extract.iloc[346, [5,6]] = [7.4, 1206.02]

extract

C:\Users\VictorMarquesMoreno\anaconda3\envs\gcb\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\VictorMarquesMoreno\AppData\Local\Temp\ipykernel_11284\247709814.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extract = extract.append(year_extract[::-1])
C:\Users\VictorMarquesMoreno\anaconda3\envs\gcb\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\VictorMarquesMoreno\AppData\Local\Temp\ipykernel_11284\247709814.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extract = extract.append(year_e

,Entrada/Saída,Movimentação,Produto,Instituição,Quantidade,Preço unitário,Valor da Operação
Data,,,,,,,
2019-11-27,Credito,Transferência - Liquidação,OIBR3,CLEAR CORRETORA - GRUPO XP,200,0.97,194.0
2019-11-27,Credito,Transferência - Liquidação,GOAU4,CLEAR CORRETORA - GRUPO XP,25,7.91,197.75
2019-11-27,Credito,Transferência - Liquidação,CIEL3,CLEAR CORRETORA - GRUPO XP,25,7.82,195.5
2019-11-29,Debito,Transferência - Liquidação,CIEL3,CLEAR CORRETORA - GRUPO XP,25,7.53,188.25
2019-12-19,Credito,Transferência - Liquidação,KNRI11,CLEAR CORRETORA - GRUPO XP,1,187.73,187.73
...,...,...,...,...,...,...,...
2022-12-14,Credito,Dividendo,CYRE3,CLEAR CORRETORA - GRUPO XP,170,0.56,96.01
2022-12-14,Credito,Rendimento,BBPO11,CLEAR CORRETORA - GRUPO XP,20,0.92,18.4
2022-12-15,Credito,Dividendo,RAIZ4,CLEAR CORRETORA - GRUPO XP,300,0.09,26.71


In [3]:
extract.Produto.unique()

array(['OIBR3', 'GOAU4', 'CIEL3', 'KNRI11', 'HGLG11', 'KLBN4', 'FLRY3',
       'SQIA3', 'EMBR3', 'ITSA4', 'POMO4', 'VVAR3', 'PETR4', 'BBPO11',
       'MGLU3', 'TRPL4', 'ENBR3', 'PMAM3', 'IRBR3', 'JBSS3', 'CYRE3',
       'HFOF11', 'HGLG12', 'IRBR1', 'BBAS3', 'BBDC4', 'MALL11', 'HFOF12',
       'COGN3', 'CPLE6', 'RAIZ4', 'VIIA3', 'LUPA3', 'OIBR1'], dtype=object)

# 1 - Transations

In [4]:
## Select only Transations
trans_data = extract[(extract['Movimentação'] == 'Transferência - Liquidação') | (extract['Movimentação'] == 'Bonificação em Ativos')]

## Auxiliar Data
stocks = trans_data.Produto.unique()
attributes = ['N of Stocks', 'Total Wealth', 'Avg Price', 'Profit']
initial_date = trans_data.index[0]+timedelta(-1)
final_date = trans_data.index[-1]
date_range = pd.date_range(initial_date, final_date)

## Organize Data by Stocks
data_idx = pd.MultiIndex.from_product([stocks, attributes])
data = pd.DataFrame(index=data_idx, columns=date_range).fillna(0)

for date in tqdm(date_range[1:]):
    
    daily_data = trans_data[trans_data.index == date]
    aux_data = daily_data.groupby(['Produto', 'Entrada/Saída']).agg(['sum', 'first'])
    aux_data = aux_data.reset_index()
    aux_data = aux_data.drop([('Movimentação','sum'), ('Instituição','sum'), ('Quantidade','first'), ('Preço unitário','sum'), ('Valor da Operação','first')], axis=1)
    aux_data = aux_data.droplevel(1, 1)
    
    data.loc[:, date] = data.loc[:, date+timedelta(-1)]
    data.loc[:, date].loc[:, 'Profit'] = 0
    
    for _, stock_data in aux_data.iterrows():
        stock = stock_data.Produto[:6].strip()
        
        if stock_data['Movimentação'] == 'Transferência - Liquidação':
            if stock_data['Entrada/Saída'] == 'Credito':
                data.loc[(stock, 'N of Stocks'), date] += stock_data['Quantidade']
                data.loc[(stock, 'Total Wealth'), date] += stock_data['Valor da Operação']
                data.loc[(stock, 'Avg Price'), date] = data.loc[(stock, 'Total Wealth'), date] / data.loc[(stock, 'N of Stocks'), date]
            elif stock_data['Entrada/Saída'] == 'Debito':
                data.loc[(stock, 'N of Stocks'), date] += - stock_data['Quantidade']
                data.loc[(stock, 'Total Wealth'), date] = data.loc[(stock, 'Avg Price'), date] * data.loc[(stock, 'N of Stocks'), date]
                data.loc[(stock, 'Profit'), date] = stock_data['Valor da Operação'] - data.loc[(stock, 'Avg Price'), date] * stock_data['Quantidade']
        elif stock_data['Movimentação'] == 'Bonificação em Ativos':
            data.loc[(stock, 'N of Stocks'), date] += stock_data['Quantidade']
            data.loc[(stock, 'Total Wealth'), date] += stock_data['Quantidade'] * data.loc[(stock, 'Avg Price'), date]
            
data

  0%|          | 0/1114 [00:00<?, ?it/s]C:\Users\VictorMarquesMoreno\AppData\Local\Temp\ipykernel_11284\811626035.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[:, date].loc[:, 'Profit'] = 0
C:\Users\VictorMarquesMoreno\AppData\Local\Temp\ipykernel_11284\811626035.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[:, date].loc[:, 'Profit'] = 0
C:\Users\VictorMarquesMoreno\AppData\Local\Temp\ipykernel_11284\811626035.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stabl

C:\Users\VictorMarquesMoreno\AppData\Local\Temp\ipykernel_11284\811626035.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[:, date].loc[:, 'Profit'] = 0
  3%|▎         | 32/1114 [00:00<00:10, 98.79it/s]C:\Users\VictorMarquesMoreno\AppData\Local\Temp\ipykernel_11284\811626035.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[:, date].loc[:, 'Profit'] = 0
C:\Users\VictorMarquesMoreno\AppData\Local\Temp\ipykernel_11284\811626035.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-d

C:\Users\VictorMarquesMoreno\AppData\Local\Temp\ipykernel_11284\811626035.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[:, date].loc[:, 'Profit'] = 0
  5%|▍         | 55/1114 [00:00<00:10, 105.84it/s]C:\Users\VictorMarquesMoreno\AppData\Local\Temp\ipykernel_11284\811626035.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[:, date].loc[:, 'Profit'] = 0
C:\Users\VictorMarquesMoreno\AppData\Local\Temp\ipykernel_11284\811626035.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-

  7%|▋         | 77/1114 [00:00<00:10, 101.86it/s]C:\Users\VictorMarquesMoreno\AppData\Local\Temp\ipykernel_11284\811626035.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[:, date].loc[:, 'Profit'] = 0
C:\Users\VictorMarquesMoreno\AppData\Local\Temp\ipykernel_11284\811626035.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[:, date].loc[:, 'Profit'] = 0
C:\Users\VictorMarquesMoreno\AppData\Local\Temp\ipykernel_11284\811626035.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-

C:\Users\VictorMarquesMoreno\AppData\Local\Temp\ipykernel_11284\811626035.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[:, date].loc[:, 'Profit'] = 0
C:\Users\VictorMarquesMoreno\AppData\Local\Temp\ipykernel_11284\811626035.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[:, date].loc[:, 'Profit'] = 0
C:\Users\VictorMarquesMoreno\AppData\Local\Temp\ipykernel_11284\811626035.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-v

...

 92%|█████████▏| 1021/1114 [00:13<00:01, 52.21it/s]C:\Users\VictorMarquesMoreno\AppData\Local\Temp\ipykernel_11284\811626035.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[:, date].loc[:, 'Profit'] = 0
C:\Users\VictorMarquesMoreno\AppData\Local\Temp\ipykernel_11284\811626035.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[:, date].loc[:, 'Profit'] = 0
C:\Users\VictorMarquesMoreno\AppData\Local\Temp\ipykernel_11284\811626035.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas

 94%|█████████▍| 1048/1114 [00:14<00:01, 57.01it/s]C:\Users\VictorMarquesMoreno\AppData\Local\Temp\ipykernel_11284\811626035.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[:, date].loc[:, 'Profit'] = 0
C:\Users\VictorMarquesMoreno\AppData\Local\Temp\ipykernel_11284\811626035.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[:, date].loc[:, 'Profit'] = 0
C:\Users\VictorMarquesMoreno\AppData\Local\Temp\ipykernel_11284\811626035.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas

 96%|█████████▌| 1072/1114 [00:14<00:00, 51.10it/s]C:\Users\VictorMarquesMoreno\AppData\Local\Temp\ipykernel_11284\811626035.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[:, date].loc[:, 'Profit'] = 0
C:\Users\VictorMarquesMoreno\AppData\Local\Temp\ipykernel_11284\811626035.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[:, date].loc[:, 'Profit'] = 0
C:\Users\VictorMarquesMoreno\AppData\Local\Temp\ipykernel_11284\811626035.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas

 98%|█████████▊| 1096/1114 [00:15<00:00, 49.53it/s]C:\Users\VictorMarquesMoreno\AppData\Local\Temp\ipykernel_11284\811626035.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[:, date].loc[:, 'Profit'] = 0
C:\Users\VictorMarquesMoreno\AppData\Local\Temp\ipykernel_11284\811626035.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[:, date].loc[:, 'Profit'] = 0
C:\Users\VictorMarquesMoreno\AppData\Local\Temp\ipykernel_11284\811626035.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas

2019-11-26  2019-11-27  2019-11-28  2019-11-29  \
OIBR3 N of Stocks            0      200.00      200.00      200.00   
      Total Wealth           0      194.00      194.00      194.00   
      Avg Price              0        0.97        0.97        0.97   
      Profit                 0        0.00        0.00        0.00   
GOAU4 N of Stocks            0       25.00       25.00       25.00   
...                        ...         ...         ...         ...   
OIBR1 Profit                 0        0.00        0.00        0.00   
VIIA3 N of Stocks            0        0.00        0.00        0.00   
      Total Wealth           0        0.00        0.00        0.00   
      Avg Price              0        0.00        0.00        0.00   
      Profit                 0        0.00        0.00        0.00   

                    2019-11-30  2019-12-01  2019-12-02  2019-12-03  \
OIBR3 N of Stocks       200.00      200.00      200.00      200.00   
      Total Wealth      194.00      194.00      194.00      194.00   
      Avg Price           0.97        0.97        0.97        0.97   
      Profit              0.00        0.00        0.00        0.00   
GOAU4 N of Stocks        25.00       25.00       25.00       25.00   
...                        ...         ...         ...         ...   
OIBR1 Profit              0.00        0.00        0.00        0.00   
VIIA3 N of Stocks         0.00        0.00        0.00        0.00   
      Total Wealth        0.00        0.00        0.00        0.00   
      Avg Price           0.00        0.00        0.00        0.00   
      Profit              0.00        0.00        0.00        0.00   

                    2019-12-04  2019-12-05  ...   2022-12-05   2022-12-06  \
OIBR3 N of Stocks       200.00      200.00  ...  3000.000000  3000.000000   
      Total Wealth      194.00      194.00  ...  4622.000000  4622.000000   
      Avg Price           0.97        0.97  ...     1.540667     1.540667   
      Profit              0.00        0.00  ...     0.000000     0.000000   
GOAU4 N of Stocks        25.00       25.00  ...   125.000000   125.000000   
...                        ...         ...  ...          ...          ...   
OIBR1 Profit              0.00        0.00  ...     0.000000     0.000000   
VIIA3 N of Stocks         0.00        0.00  ...  1353.000000  1353.000000   
      Total Wealth        0.00        0.00  ...  4156.170000  4156.170000   
      Avg Price           0.00        0.00  ...     3.071818     3.071818   
      Profit              0.00        0.00  ...     0.000000     0.000000   

                     2022-12-07   2022-12-08   2022-12-09   2022-12-10  \
OIBR3 N of Stocks   3000.000000  3000.000000  3000.000000  3000.000000   
      Total Wealth  4622.000000  4622.000000  4622.000000  4622.000000   
      Avg Price        1.540667     1.540667     1.540667     1.540667   
      Profit           0.000000     0.000000     0.000000     0.000000   
GOAU4 N of Stocks    125.000000   125.000000   125.000000   125.000000   
...                         ...          ...          ...          ...   
OIBR1 Profit           0.000000     0.000000     0.000000     0.000000   
VIIA3 N of Stocks   1353.000000  1353.000000  1353.000000  1353.000000   
      Total Wealth  4156.170000  4156.170000  4156.170000  4156.170000   
      Avg Price        3.071818     3.071818     3.071818     3.071818   
      Profit           0.000000     0.000000     0.000000     0.000000   

                     2022-12-11   2022-12-12   2022-12-13   2022-12-14  
OIBR3 N of Stocks   3000.000000  3000.000000  3000.000000  3000.000000  
      Total Wealth  4622.000000  4622.000000  4622.000000  4622.000000  
      Avg Price        1.540667     1.540667     1.540667     1.540667  
      Profit           0.000000     0.000000     0.000000     0.000000  
GOAU4 N of Stocks    125.000000   125.000000    75.000000    75.000000  
...                         ...          ...          ...          ...  
OIBR1 Profit 

## 1.1 - Profit and Prejudice

In [5]:
operation = data.loc[(slice(None), 'Profit'), :]
operation = operation.droplevel(1, 0).transpose()

## Reference Year
operation = operation[operation.index.year == year]
operation.index = operation.index.month

profit = operation[operation >= 0]
prejudice = operation[operation <= 0]

profit = profit.groupby(profit.index).sum()
prejudice = prejudice.groupby(prejudice.index).sum()

print(profit.sum(axis=1))
print(profit.sum(axis=1).sum())

print(prejudice.sum(axis=1))
print(prejudice.sum(axis=1).sum())

print(profit.sum(axis=1) + prejudice.sum(axis=1))

1      266.000000
2        0.000000
3      490.471667
4        0.000000
5     1802.623691
6        0.000000
7        0.000000
8      970.957978
9      188.177645
10    1208.414195
11     252.517086
12     563.794634
dtype: float64
5742.956895683861
1       0.000000
2       0.000000
3       0.000000
4       0.000000
5       0.000000
6       0.000000
7       0.000000
8     -92.640000
9    -504.972225
10   -149.990000
11      0.000000
12   -228.188571
dtype: float64
-975.7907965747704
1      266.000000
2        0.000000
3      490.471667
4        0.000000
5     1802.623691
6        0.000000
7        0.000000
8      878.317978
9     -316.794580
10    1058.424195
11     252.517086
12     335.606063
dtype: float64


# 2 - Position

In [6]:
aux_data = data.iloc[:, -1].reset_index()
position = pd.pivot_table(aux_data, values=data.columns[-1], index='level_0', columns='level_1')
position = position.drop('Profit', axis=1)
position[position['N of Stocks'] != 0]

level_1,Avg Price,N of Stocks,Total Wealth
level_0,,,
BBDC4,21.587445,175.0,3777.802788
BBPO11,103.236000,20.0,2064.720000
CYRE3,16.762800,100.0,1676.280000
GOAU4,9.091088,75.0,681.831620
HFOF11,95.468750,24.0,2291.250000
HGLG11,172.637556,10.0,1726.375556
IRBR1,0.000000,-16.0,0.000000
ITSA4,10.229804,115.0,1176.427487
JBSS3,21.198298,47.0,996.320000


In [7]:
x = extract[(extract.Produto == 'IRBR1')]
x['Cum Qnt'] = copy.deepcopy(x['Quantidade'])
aux = 0
for idx in range(len(x)):
    if x.iloc[idx, 1] in ['Transferência - Liquidação', 'Bonificação em Ativos']:
        if x.iloc[idx, 0] == 'Credito':
            aux += x.iloc[idx, 4]
        elif x.iloc[idx, 0] == 'Debito':
            aux -= x.iloc[idx, 4]
        
    x.iloc[idx, 7] = aux
        
x

C:\Users\VictorMarquesMoreno\AppData\Local\Temp\ipykernel_11284\3578035214.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x['Cum Qnt'] = copy.deepcopy(x['Quantidade'])


,Entrada/Saída,Movimentação,Produto,Instituição,Quantidade,Preço unitário,Valor da Operação,Cum Qnt
Data,,,,,,,,
2020-07-15,Credito,Direito de Subscrição,IRBR1,CLEAR CORRETORA - GRUPO XP,16,-,-,0
2020-08-10,Debito,Transferência - Liquidação,IRBR1,CLEAR CORRETORA - GRUPO XP,16,0.43,6.88,-16
2020-08-12,Debito,Direitos de Subscrição - Não Exercido,IRBR1,CLEAR CORRETORA - GRUPO XP,0,-,-,-16


# 3 - Earnings

## 3.1 -  Dividends

In [8]:
dividends = extract[(extract['Movimentação'] == 'Dividendo') | (extract['Movimentação'] == 'Rendimento')]
dividends = dividends.drop(['Entrada/Saída', 'Movimentação', 'Instituição', 'Quantidade', 'Preço unitário'], axis=1)
dividends.groupby('Produto').sum()

,Valor da Operação
Produto,
BBAS3,69.82
BBDC4,34.60
BBPO11,311.21
CPLE6,194.30
CYRE3,128.52
ENBR3,268.96
FLRY3,49.36
GOAU4,672.58
HFOF11,272.49


## 3.2 - JCP

In [9]:
JCP = extract[extract['Movimentação'] == 'Juros Sobre Capital Próprio']
JCP = JCP.drop(['Entrada/Saída', 'Movimentação', 'Instituição', 'Quantidade', 'Preço unitário'], axis=1)
JCP.reset_index().groupby(['Produto']).sum('Valor da Operação')

""
Produto
BBAS3
BBDC4
CIEL3
CPLE6
ENBR3
FLRY3
GOAU4
IRBR3
ITSA4


In [10]:
extract['Movimentação'].unique()

array(['Transferência - Liquidação', 'Rendimento',
       'Juros Sobre Capital Próprio', 'Dividendo',
       'Direito de Subscrição', 'Cessão de Direitos - Solicitada',
       'Cessão de Direitos', 'Direitos de Subscrição - Não Exercido',
       'Atualização', 'Bonificação em Ativos', 'Fração em Ativos',
       'Leilão de Fração', 'Transferência', 'Rendimento - Cancelado',
       'Dividendo - Cancelado'], dtype=object)